In [13]:
import gensim.downloader
import pandas
import numpy as np

In [14]:
data = pandas.read_csv('synonyms.csv', sep=",",header=0)
data

,question,answer,0,1,2,3
0,enormously,tremendously,appropriately,uniquely,tremendously,decidedly
1,provisions,stipulations,stipulations,interrelations,jurisdictions,interpretations
2,haphazardly,randomly,dangerously,densely,randomly,linearly
3,prominent,conspicuous,battered,ancient,mysterious,conspicuous
4,zenith,pinnacle,completion,pinnacle,outset,decline
...,...,...,...,...,...,...
75,fashion,manner,ration,fathom,craze,manner
76,marketed,sold,frozen,sold,sweetened,diluted
77,bigger,larger,steadier,closer,larger,better
78,roots,origins,origins,rituals,cure,function


In [15]:
model = 'word2vec-google-news-300'
word2vec = gensim.downloader.load(model)

In [16]:
L_pred = []
for row in data.iterrows():
    if row[1]["question"] in word2vec.key_to_index and row[1]["answer"] in word2vec.key_to_index:
        prediction = []
        if row[1]["0"] in word2vec.key_to_index:
            prediction.append([row[1]['0'], word2vec.similarity(row[1]['0'], row[1]['question'])]) 
            
        if row[1]["1"] in word2vec.key_to_index:
            prediction.append([row[1]['1'], word2vec.similarity(row[1]['1'], row[1]['question'])])
            
        if row[1]["2"] in word2vec.key_to_index:
            prediction.append([row[1]['2'], word2vec.similarity(row[1]['2'], row[1]['question'])])
            
        if row[1]["3"] in word2vec.key_to_index:
            prediction.append([row[1]['3'], word2vec.similarity(row[1]['3'], row[1]['question'])])
        
        if len(prediction)!=0:
            guess = max(prediction, key=lambda x: x[1])
            if guess[0] == row[1]["answer"]:
                label='correct'
            else:
                label='wrong'
        else:
            label = 'guess'
            guess = [row[1][str(np.random.randint(0,4))]]
            
    else:
        label = 'guess'
        guess = [row[1][str(np.random.randint(0,4))]]
    s = F"{row[1]['question']},{row[1]['answer']},{guess[0]},{label}"
    L_pred.append(s)

with open(f"{model}-details.csv",'w') as f:
    for pred in L_pred:
        f.write(pred+'\n')
        print(pred)

enormously,tremendously,tremendously,correct
provisions,stipulations,stipulations,correct
haphazardly,randomly,randomly,correct
prominent,conspicuous,conspicuous,correct
zenith,pinnacle,pinnacle,correct
flawed,imperfect,imperfect,correct
urgently,desperately,desperately,correct
consumed,eaten,eaten,correct
advent,coming,coming,correct
concisely,succinctly,succinctly,correct
salutes,greetings,ceremonies,wrong
solitary,alone,restless,wrong
hasten,accelerate,accelerate,correct
perseverance,endurance,generosity,wrong
fanciful,imaginative,imaginative,correct
showed,demonstrated,demonstrated,correct
constantly,continually,continually,correct
issues,subjects,subjects,correct
furnish,supply,impress,wrong
costly,expensive,expensive,correct
recognized,acknowledged,acknowledged,correct
spot,location,location,correct
make,earn,earn,correct
often,frequently,frequently,correct
easygoing,relaxed,relaxed,correct
debate,argument,argument,correct
narrow,thin,thin,correct
arranged,planned,planned,correct

In [17]:
L_pred = []
with open(f"{model}-details.csv",'r') as f:
    for line in f.readlines():
        L_pred.append(line.split(','))
        L_pred[-1][-1] = L_pred[-1][-1].rstrip("\n")
C = list(map(lambda x:x[-1],L_pred)).count("correct")
V = list(map(lambda x:x[-1],L_pred)).count("wrong") + C

with open("analysis.csv", "w") as f:
    f.write(f"{model},{len(word2vec.key_to_index)},{C},{V},{C/V}\n")
    print(f"{model},{len(word2vec.key_to_index)},{C},{V},{C/V}")

word2vec-google-news-300,3000000,70,79,0.8860759493670886
